In [26]:
import pandas as pd

In [27]:
tsla_df = pd.read_csv('data/tsla_df_dummies.csv')
tsla_df

,Date,Close,High,Low,Open,Volume,date_weekday,netChangeRatio,positive
0,2020-01-02,28.684000,28.713333,28.114000,28.299999,142981500,3,NaN,0
1,2020-01-03,29.534000,30.266666,29.128000,29.366667,266677500,4,2.963326,1
2,2020-01-06,30.102667,30.104000,29.333332,29.364668,151995000,0,1.925464,1
3,2020-01-07,31.270666,31.441999,30.224001,30.760000,268231500,1,3.880052,1
4,2020-01-08,32.809334,33.232666,31.215334,31.580000,467164500,2,4.920483,1
...,...,...,...,...,...,...,...,...,...
500,2021-12-27,364.646667,372.333344,356.906677,357.890015,71145900,0,2.524839,1
501,2021-12-28,362.823334,373.000000,359.473328,369.829987,60324000,1,-0.500028,0
502,2021-12-29,362.063324,368.000000,354.713318,366.213318,56154000,2,-0.209471,0
503,2021-12-30,356.779999,365.183319,351.049988,353.776672,47040900,3,-1.459227,0


In [28]:
# 定義跳空圈口的閾值，這裡設定成中位數的3%
jump_threshold = tsla_df.Close.median() * 0.03
print(jump_threshold)

# 補一下preClouse欄位
tsla_df['preClose'] = tsla_df['Close'].shift(1)

# 函數計算跳空方向（漲1跌-1)與跳空能量
def detect_jump(row):
    # 初始化
    jump = 0
    jump_power = 0
    
    # 向上跳空
    if row['netChangeRatio'] > 0 and (row['Low'] - row['preClose']) > jump_threshold:
        jump = 1
        jump_power = (row['Low'] - row['preClose']) / jump_threshold
    
    # 向下跳空
    elif row['netChangeRatio'] < 0 and (row['preClose'] - row['High']) > jump_threshold:
        jump = -1
        jump_power = (row['preClose'] - row['High']) / jump_threshold
        
    return pd.Series([jump, jump_power])


# 套用
tsla_df[['jump', 'jump_power']] = tsla_df.apply(detect_jump, axis=1)

# 只保留真的有跳空的資料
jump_df = tsla_df[tsla_df['jump'] != 0][['Date', 'Close', 'netChangeRatio','preClose','jump','jump_power']]

jump_df.head()

6.048700103759765


,Date,Close,netChangeRatio,preClose,jump,jump_power
172,2020-09-08,110.070000,-21.062824,139.440002,-1.0,2.732268
222,2020-11-17,147.203339,8.213879,136.029999,1.0,1.373299
245,2020-12-21,216.619995,-6.494968,231.666672,-1.0,1.460371
256,2021-01-07,272.013336,7.944656,251.993332,1.0,1.059180
257,2021-01-08,293.339996,7.840299,272.013336,1.0,1.231667
